In [1]:
%load_ext autoreload
%autoreload 2

In [21]:

from Libs.Dataset import Dataset
from Libs.Model import Net
from Libs.train import train_model
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import csv
np.random.seed(0)
torch.manual_seed(0)

In [3]:
BATCH_SIZE = 4

In [4]:
dir_training = "./data/training_data/training_data"
dir_testing = "./data//testing_data/testing_data"
csv_file = "data/training_labels.csv"

In [15]:

df = pd.read_csv(csv_file)
label_ids = {}
for label in df["label"]:
    if label not in label_ids:
        label_ids[label] = len(label_ids)
id_labels =  {v: k for k, v in label_ids.items()}

## Dataset preprocess

In [6]:
# data.train = data.load_dir(dir_training)
# data.test = data.load_dir(dir_testing)
train_trans = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                                 transforms.Normalize((0.5), (0.5))])
# train_dataset = torchvision.datasets.ImageFolder(root=dir_training, transform=train_trans)
train_dataset = Dataset(dir_training, csv_file, label_ids, transform=train_trans)
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True,drop_last=True)
test_trans = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                                 transforms.Normalize((0.5), (0.5))])
test_dataset =  Dataset(dir_testing, csv_file, label_ids=None, transform=test_trans)                               
# test_dataset = torchvision.datasets.ImageFolder(root=dir_training, transform=test_trans)                                 
test_loader =  torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last=True)



## Model

In [7]:
model = Net()
model.to(model.device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
import traceback

model, training_losses, training_accs, test_accs = train_model(model, train_loader, test_loader, criterion, optimizer, lrscheduler, 10)


Training epoch 1 ...
Epoch 1, duration: 2 s, loss: 0.2236, acc: 0.8200
Training epoch 2 ...
Epoch 2, duration: 2 s, loss: 0.3503, acc: 0.7100
Training epoch 3 ...
Epoch 3, duration: 2 s, loss: 0.2041, acc: 0.6800
Training epoch 4 ...
Epoch 4, duration: 2 s, loss: 0.1200, acc: 0.7900
Training epoch 5 ...
Epoch 5, duration: 2 s, loss: 0.0912, acc: 0.8400
Training epoch 6 ...
Epoch 6, duration: 2 s, loss: 0.1005, acc: 0.8300
Training epoch 7 ...
Epoch 7, duration: 2 s, loss: 0.0736, acc: 0.9000
Training epoch 8 ...
Epoch 8, duration: 2 s, loss: 0.0859, acc: 0.8800
Training epoch 9 ...
Epoch 9, duration: 2 s, loss: 0.0560, acc: 0.9400
Training epoch 10 ...
Epoch 10, duration: 2 s, loss: 0.0781, acc: 0.8900
Finished Training


## Predict

In [24]:
predict = {}
model.eval()
with torch.no_grad():
    
    for inputs in test_loader:
        imgs, img_names = inputs
        imgs = imgs.to(model.device)
        p = model.predict(imgs)
        for i, img_name in enumerate(img_names):
            predict[img_name] = id_labels[p[i].item()]
    # print(predict)

with open('submission.csv', 'w',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["id", "labels"])
    writer.writeheader()
    for key, value in predict.items():
        writer.writerow({'id': key, 'labels': value})
print("Done")
    

Done
